# Investigator Agent - Data Review Branch Example

This notebook provides a complete, self-contained definition of the **Data Review** agent branch.
It includes the definitions for tools, sub-agent prompts, sub-agents, and the main Data Review agent.

This allows for visual inspection of the entire logic flow in one place.

In [ ]:
import os
from dotenv import load_dotenv
from google.cloud import bigquery
from google.adk import Agent
from google.adk.tools import google_search
from google.adk.tools.agent_tool import AgentTool
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Load environment variables
load_dotenv()

## Tool Definitions

### Weather Check Tool (Parameterized SQL)
This tool queries BigQuery public data (`noaa_gsod`) to verify weather events.

In [ ]:
def check_weather_event(
    state: str, year: str, month: str, day: str, event_type: str
) -> dict:
    """
    Checks if a specific weather event occurred in a given state on a specific date.

    Args:
        state: 2-letter state code (e.g., 'TN').
        year: 4-digit year (e.g., '1991').
        month: Month (e.g., '07').
        day: Day (e.g., '04').
        event_type: One of: fog, rain, snow, hail, thunder, tornado.
    """
    client = bigquery.Client()

    # Map user-friendly event names to actual BQ column names
    schema_map = {
        "fog": "fog",
        "rain": "rain_drizzle",
        "snow": "snow_ice_pellets",
        "hail": "hail",
        "thunder": "thunder",
        "tornado": "tornado_funnel_cloud",
    }

    # Validate the event_type to prevent SQL injection since we inject the column name
    if event_type not in schema_map:
        return {
            "error": f"Invalid event_type. Must be one of {list(schema_map.keys())}"
        }

    target_column = schema_map[event_type]
    table_id = f"bigquery-public-data.noaa_gsod.gsod{year}"

    query = f"""
        SELECT COUNT(*) as event_count
        FROM `{table_id}` y
        WHERE y.stn IN (
            SELECT s.usaf 
            FROM `bigquery-public-data.noaa_gsod.stations` s 
            WHERE state = @state
        )
        AND y.da = @day
        AND y.mo = @month
        AND y.{target_column} = "1"
    """

    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("state", "STRING", state),
            bigquery.ScalarQueryParameter("day", "STRING", day),
            bigquery.ScalarQueryParameter("month", "STRING", month),
        ]
    )

    print(f"DEBUG: Executing SQL Query:\n{query}")
    print(
        f"DEBUG: Parameters: state={state}, day={day}, month={month}, event_type={event_type}"
    )

    try:
        query_job = client.query(query, job_config=job_config)
        results = query_job.result()

        # Get the first row
        row = next(iter(results))
        found = row.event_count > 0

        print(f"DEBUG: SQL Result: event_count={row.event_count}, found={found}")

        return {
            "event_occurred": found,
            "station_count": row.event_count,
            "details": f"Found {row.event_count} stations in {state} reporting {event_type} on {month}/{day}/{year}.",
        }
    except Exception as e:
        print(f"DEBUG: SQL Error: {e}")
        return {"error": str(e)}

## Sub-agent 1: SQL Reviewer (Weather)

This sub-agent uses the `check_weather_event` tool.

In [ ]:
SQL_REVIEWER_PROMPT = """You are a SQL Data Reviewer specializing in weather events.
Your goal is to check if specific weather events occurred using BigQuery public data.
Use the `check_weather_event` tool to verify weather conditions.
Analyze the results and provide a summary of the findings.
"""

SQL_REVIEWER_DESCRIPTION = "Checks for weather events using SQL queries."

sql_reviewer = Agent(
    name="sql_reviewer",
    model="gemini-2.5-flash",
    description=SQL_REVIEWER_DESCRIPTION,
    instruction=SQL_REVIEWER_PROMPT,
    tools=[check_weather_event],
)

## Sub-agent 2: Search Reviewer

This sub-agent uses the `google_search` tool.

In [ ]:
SEARCH_REVIEWER_PROMPT = """You are a Search Reviewer.
Your goal is to answer questions unrelated to the SQL data by performing a Google Search.
Use the `google_search` tool to find information.
Analyze the search results and provide a comprehensive answer.
"""

SEARCH_REVIEWER_DESCRIPTION = "Performs Google searches to answer questions."

search_reviewer = Agent(
    name="search_reviewer",
    model="gemini-2.5-flash",
    description=SEARCH_REVIEWER_DESCRIPTION,
    instruction=SEARCH_REVIEWER_PROMPT,
    tools=[google_search],
)

## Level 1 Agent: Data Review

This agent orchestrates the two sub-agents.

In [ ]:
DATA_REVIEW_PROMPT = """You are a Data Review Agent.
Your goal is to answer data-related questions by delegating to your sub-agents.

If the question involves checking for weather events or querying weather data, delegate to the `sql_reviewer`.
If the question requires searching the web for information not in the database, delegate to the `search_reviewer`.

Synthesize the information provided by your sub-agents to answer the user's question.
"""

data_review = Agent(
    name="data_review",
    model="gemini-2.5-flash",
    description="You are an expert in Data Review.",
    instruction=DATA_REVIEW_PROMPT,
    tools=[
        AgentTool(agent=sql_reviewer),
        AgentTool(agent=search_reviewer),
    ],
)

print("Data Review Agent Initialized.")

## Usage Examples

Below are examples of how to interact with the agent using the ADK Runner.
Note the use of `await` for async operations.

In [ ]:
from google.adk.agents import Agent
from vertexai.agent_engines import AdkApp

app = AdkApp(agent=data_review)

async def call_agent(app, query):
    async for event in app.async_stream_query(
       user_id="u_1234",  # Required
       message=query,
    ):
       print(event)

In [ ]:
# Example 1: Weather Query
await call_agent(app, "Did it rain in Rhode Island on July 4th 1990?")

In [ ]:
# Example 2: General Query
await call_agent(app, "What is the capital of France?")